In [1]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from scipy.sparse import csr_matrix
import time

In [3]:
products = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('products.csv')

dfMerged = pd.merge(orders, products, on="product_id", how="inner")

# Agrupar productos por transacción para crear listas de transacciones

transactions = dfMerged.groupby("order_id")["product_name"].apply(list).tolist()
print(f"Primeras transacciones:\n{transactions[:5]}")

item_mapping = {item: idx for idx, item in enumerate(sorted(set(item for transaction in transactions for item in transaction)))}
rows, cols = [], []
for row_idx, transaction in enumerate(transactions):
    for item in transaction:
        rows.append(row_idx)
        cols.append(item_mapping[item])

# Crear la matriz dispersa
order_matrix_sparse = csr_matrix(([1] * len(rows), (rows, cols)), shape=(len(transactions), len(item_mapping)))

Primeras transacciones:
[['Organic Celery Hearts', 'Organic 4% Milk Fat Whole Milk Cottage Cheese', 'Bag of Organic Bananas', 'Organic Whole String Cheese', 'Lightly Smoked Sardines in Olive Oil', 'Organic Hass Avocado', 'Bulgarian Yogurt', 'Cucumber Kirby'], ['Spring Water', 'Prosciutto, Americano', 'Grated Pecorino Romano Cheese', 'Super Greens Salad', 'Cage Free Extra Large Grade AA Eggs', 'Asparagus', 'Organic Garnet Sweet Potato (Yam)', 'Organic Half & Half'], ['Organic Raw Unfiltered Apple Cider Vinegar', 'Shelled Pistachios', 'Organic Biologique Limes', 'Organic Baby Arugula', 'Organic Hot House Tomato', 'Bunched Cilantro', 'Green Peas', 'Fresh Dill', 'Flat Parsley, Bunch'], ['Roasted Turkey', 'Organic Whole Strawberries', 'Organic Pomegranate Kernels', 'Organic Raspberries', 'Organic Cucumber', 'Organic Blueberries', 'Organic Grape Tomatoes'], ['Organic Whole Grassmilk Milk', 'Garbanzo Beans', 'Geranium Liquid Dish Soap', 'Corn Maize Tortillas', 'Organic Chocolate Almondmilk Pu

In [11]:
# Función para calcular soporte de elementos individuales
#def calculate_support(item_indices, data_matrix):
    
    #item_mask = data_matrix[:, item_indices].toarray().all(axis=1)
    #support = np.sum(item_mask) / data_matrix.shape[0]
    #return support

#!DESCOMENTAR EN CASO DE NO FUNCIONARLA FUNCION DE ARRIBA (KEVIN A.)

In [4]:
#Mejora de funcion para calcular el soporte V1.2
def calculate_support(item_indices, data_matrix):
    #para un solo indice, optimizamos el calculo
    if isinstance(item_indices, int) or (hasattr(item_indices, '__len__') and len((item_indices) == 1)):
        if hasattr(item_indices, '__len__'):
            idx = item_indices[0]
        else:
            idx = item_indices
            #Contamos las ocurrencias de la columna 
        col_sum = data_matrix.getcol(idx).sum()
        return float(col_sum) / data_matrix.shape[0]
    
    #Ahora para multiples indices, operamos conjuntos en las filas 
    row_with_items = set(range(data_matrix.shape[0]))
    for idx in  item_indices:
        #Obtenemos los indices de filas donde aparece este item
        col = data_matrix.getcol(idx)
        rows_with_time = set(col.nonzero()[0])
        #interseccion con filas que contienen todos los items anteriores 
        row_with_items &= rows_with_item
    
    support = len(rows_with_items) / data_matrix.shape[0]
    return support 

In [5]:
# Función para generar ítems frecuentes utilizando matrices dispersas
def apriori_manual(data_matrix, min_support):
  
    num_items = data_matrix.shape[1]
    frequent_itemsets = []
    current_itemsets = [[i] for i in range(num_items)]
    
    while current_itemsets:
        next_itemsets = []
        item_supports = []
        
        # Calcular soporte para cada conjunto actual
        for itemset in current_itemsets:
            support = calculate_support(itemset, data_matrix)
            if support >= min_support:
                frequent_itemsets.append((itemset, support))
                item_supports.append(itemset)
        
        # Generar nuevas combinaciones de ítems frecuentes actuales
        for i in range(len(item_supports)):
            for j in range(i + 1, len(item_supports)):
                combined_itemset = sorted(set(item_supports[i]) | set(item_supports[j]))
                if len(combined_itemset) == len(item_supports[i]) + 1:
                    next_itemsets.append(combined_itemset)
        
        current_itemsets = next_itemsets  # Actualizar conjuntos actuales
    
    return frequent_itemsets

In [6]:
#Convertimos la matriz dispersa a una lista de listas 
def sparse_transaction(data_matrix, item_mapping):
    reverse_mapping = {idx: item for item, idx in item_mapping.items()}

    transactions = []
    #Aplicamos la función a cada fila de la matriz dispersa
    for i in range(data_matrix.shape[0]):
        #obtener indices de elementos no cero
        row = data_matrix[i]
        row_indices = row.indices
        
        # Convertir índices a nombres de productos usando el mapeo inverso
        transaction = []
        for idx in row_indices:
            if idx in reverse_mapping:
                transaction.append(reverse_mapping[idx])
            else:
                print(f"Advertencia: Índice {idx} no encontrado en el mapeo inverso")
        
        transactions.append(transaction)
    
    return transactions



In [ ]:
#Primera version (Esta funcion no es muy optima)
# def generar_candidatos(itemsets_frecuentes_k, k,max_candidates= 100000):
    # if len(itemsets_frecuentes_k) > 1000:
    #     print(f"Demasiados itemsets frecuentes ({len(itemsets_frecuentes_k)}). Limitando a los 1000 más frecuentes.")
    #     # Aquí asumimos que los itemsets ya están ordenados por soporte
    #     itemsets_frecuentes_k = itemsets_frecuentes_k[:1000]
    # candidatos = []
    # n = len(itemsets_frecuentes_k)
    
    # join => combinar itemsets que comparten k-1 elementos
    # for i in range(n):
    #     for j in range(i+1, n):
    #         # k=1 simplemente combinamos los items
    #         if k == 1:
    #             candidato = [itemsets_frecuentes_k[i][0], itemsets_frecuentes_k[j][0]]
    #             candidatos.append(sorted(candidato))
    #         else:
    #             # Para k>1 verificamos si los primeros k-1  son iguales
    #             if itemsets_frecuentes_k[i][:k-1] == itemsets_frecuentes_k[j][:k-1]:
    #                 # Combinamos
    #                 candidato = itemsets_frecuentes_k[i].copy()
    #                 candidato.append(itemsets_frecuentes_k[j][k-1])
    #                 candidato.sort()
    #                 candidatos.append(candidato)
    
    # # Eliminar duplicados usando un conjunto (set)
    # candidatos_unicos = [list(x) for x in set(tuple(x) for x in candidatos)]
    # return candidatos_unicos

In [7]:
#Version 1.2
def generar_candidatos(itemsets_frecuentes_k, k, max_candidates=100000, support_dict=None):
    # Limitar itemsets frecuentes si hay demasiados
    if len(itemsets_frecuentes_k) > 1000:
        print(f"Demasiados itemsets frecuentes ({len(itemsets_frecuentes_k)}). Limitando a los 1000 más frecuentes.")
        if support_dict:
            # Ordenar por soporte si tenemos el diccionario de soporte
            itemsets_frecuentes_k = sorted(itemsets_frecuentes_k, 
                                          key=lambda x: support_dict.get(tuple(x), 0), 
                                          reverse=True)[:1000]
        else:
            itemsets_frecuentes_k = itemsets_frecuentes_k[:1000]
    
    # Estimación del número de candidatos que se generarán
    n = len(itemsets_frecuentes_k)
    estimated_candidates = n * (n - 1) // 2
    
    # Si la estimación supera el máximo, aplicamos filtrado adicional
    if estimated_candidates > max_candidates:
        reduction_factor = max_candidates / estimated_candidates
        limit = int(n * reduction_factor**0.5)
        print(f"Estimación de candidatos ({estimated_candidates}) excede el máximo. Limitando a {limit} itemsets.")
        if support_dict:
            itemsets_frecuentes_k = sorted(itemsets_frecuentes_k, 
                                          key=lambda x: support_dict.get(tuple(x), 0), 
                                          reverse=True)[:limit]
        else:
            itemsets_frecuentes_k = itemsets_frecuentes_k[:limit]
        n = len(itemsets_frecuentes_k)
    
    # Usar un conjunto para evitar duplicados desde el principio
    candidatos_set = set()
    
    # Para k=1, usar un enfoque más eficiente
    if k == 1:
        for i in range(n):
            for j in range(i+1, n):
                candidato = tuple(sorted([itemsets_frecuentes_k[i][0], itemsets_frecuentes_k[j][0]]))
                candidatos_set.add(candidato)
    else:
        # Optimización: creo un índice para agrupar itemsets que comparten prefijo
        prefix_index = {}
        for idx, itemset in enumerate(itemsets_frecuentes_k):
            prefix = tuple(itemset[:k-1])
            if prefix not in prefix_index:
                prefix_index[prefix] = []
            prefix_index[prefix].append(idx)
        
        # Generar candidatos solo entre itemsets con el mismo prefijo
        for prefix, indices in prefix_index.items():
            if len(indices) > 1:  # Necesitamos al menos 2 itemsets con el mismo prefijo
                for i in range(len(indices)):
                    for j in range(i+1, len(indices)):
                        idx1, idx2 = indices[i], indices[j]
                        # Crear candidato
                        candidato = list(prefix) + [itemsets_frecuentes_k[idx1][k-1], itemsets_frecuentes_k[idx2][k-1]]
                        candidato.sort()
                        candidatos_set.add(tuple(candidato))
    
    # Convertir de vuelta a lista de listas
    candidatos_unicos = [list(x) for x in candidatos_set]
    
    # Verificar si aún tenemos demasiados candidatos
    if len(candidatos_unicos) > max_candidates:
        print(f"Aún hay demasiados candidatos ({len(candidatos_unicos)}). Limitando a {max_candidates}.")
        candidatos_unicos = candidatos_unicos[:max_candidates]
    
    return candidatos_unicos

In [8]:
def poda_apriori(candidatos, itemsets_frecuentes_k, k):
    # Convertir itemsets frecuentes a conjunto para búsqueda eficiente O
    itemsets_frecuentes_set = set(tuple(x) for x in itemsets_frecuentes_k)
    candidatos_podados = []
    
    for candidato in candidatos:
        es_valido = True
        
        # Generar todos los subconjuntos de tamaño k 
        for i in range(len(candidato)):
            subconjunto = candidato.copy()
            subconjunto.pop(i)
            
            # si algun subconunto no es frecuente , el candidato no puede ser frecuente
            if tuple(subconjunto) not in itemsets_frecuentes_set:
                es_valido = False
                break
        
        if es_valido:
            candidatos_podados.append(candidato)
    
    return candidatos_podados

In [ ]:
#Implementacion manual dde priori con procesamiento por lotes
def apriori_lotes(transactions_list, min_support, batch_size=50000):
    n_total = len(transactions_list)
    print(f"Procesando {n_total} transacciones con soporte minimo {min_support}")

    item_counts = {}

    n_batches = (n_total + batch_size - 1) // batch_size

    print('Fase 1: Generando 1-itemsets frecuentes')

    #contar items individuales
    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, n_total)
        print(f"Procesando lote {i + 1}/ {n_batches} (transacciones {start_idx}-{end_idx})")
        
        # Transacciones para este lote
        batch_transactions = transactions_list[start_idx:end_idx]

        # Contador de items individuales
        for transaction in batch_transactions:
            for item in transaction:
                item_counts[item] = item_counts.get(item, 0) + 1

    # Filtrado por soporte mínimo
    frequent_1_itemsets = []
    support_dict = {}

    for item, count in item_counts.items():
        support = count / n_total
        if support >= min_support:
            frequent_1_itemsets.append([item])
            support_dict[tuple([item])] = support
    
    print(f"  Se encontraron {len(frequent_1_itemsets)} 1-itemsets frecuentes")

    all_frequent_itemsets = {1: frequent_1_itemsets}

    k = 1
    while k in all_frequent_itemsets and all_frequent_itemsets[k]:  # Verificación más explícita
        print(f"Fase {k + 1}: Generando {k + 1}-itemsets frecuentes")

        if len(all_frequent_itemsets[k]) > 1000:
            print(f"Demasiados itemsets frecuentes ({len(all_frequent_itemsets[k])}). Limitando a los 1000 con mayor soporte.")
           
            sorted_itemsets = sorted(all_frequent_itemsets[k], 
                                    key=lambda x: support_dict[tuple(x)], 
                                    reverse=True)[:1000]
            all_frequent_itemsets[k] = sorted_itemsets
            print(f"Limitado a {len(all_frequent_itemsets[k])} itemsets frecuentes")
        
        # Generación
        candidatos = generar_candidatos(all_frequent_itemsets[k], k)

        if len(candidatos) > 100000:
            print(f"¡Demasiados candidatos ({len(candidatos)})! Aumentando el umbral de soporte para esta fase.")
            # Aumentar temporalmente el umbral de soporte
            temp_min_support = min_support * 2
            print(f"Umbral de soporte temporal: {temp_min_support}")

            # Filtrar itemsets frecuentes con el nuevo umbral
            filtered_itemsets = []
            for itemset in all_frequent_itemsets[k]:
                if support_dict[tuple(itemset)] >= temp_min_support:
                    filtered_itemsets.append(itemset)
            
            print(f"Filtrando a {len(filtered_itemsets)} itemsets con mayor soporte")
            # Regenerar candidatos con el conjunto filtrado
            candidatos = generar_candidatos(filtered_itemsets, k)
            print(f"Nuevos candidatos generados: {len(candidatos)}")
    
        # Aplicación de poda
        candidatos_podados = poda_apriori(candidatos, all_frequent_itemsets[k], k)

        print(f"Generados {len(candidatos)} candidatos, {len(candidatos_podados)} despues de poda")

        if not candidatos_podados:
            print(f"No hay candidatos después de la poda. Terminando.")
            break

        # Dic para apariciones de candidatos
        candidato_counts = {tuple(c): 0 for c in candidatos_podados}
    
        # Procesado por lotes para contar candidatos 
        for i in range(n_batches): 
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, n_total)

            print(f"Procesando lote {i+1}/{n_batches} (transacciones {start_idx}-{end_idx})")

            batch_transactions = transactions_list[start_idx:end_idx]
            
            # Precomputar los conjuntos de transacciones
            transaction_sets = [set(transaction) for transaction in batch_transactions]
            
            # Usar un enfoque más eficiente para verificar candidatos
            for t_idx, transaction_set in enumerate(transaction_sets):
                # Solo procesar transacciones que tengan suficientes elementos
                if len(transaction_set) >= k + 1:
                    for candidato in candidatos_podados:
                        # Verificación más rápida usando issubset
                        if set(candidato).issubset(transaction_set):
                            candidato_counts[tuple(candidato)] += 1
                
                # Mostrar progreso 
                if (t_idx + 1) % 10000 == 0:
                    print(f"  Progreso: {t_idx + 1}/{len(batch_transactions)} transacciones procesadas")
        
        frequent_itemsets_k_plus_1 = []
        
        for candidato, count in candidato_counts.items():
            support = count / n_total
            if support >= min_support:
                frequent_itemsets_k_plus_1.append(list(candidato))
                support_dict[candidato] = support
        
        print(f"  Se encontraron {len(frequent_itemsets_k_plus_1)} {k+1}-itemsets frecuentes")
        
        # Continua si encuentra itemsets frecuentes
        if frequent_itemsets_k_plus_1:
            all_frequent_itemsets[k+1] = frequent_itemsets_k_plus_1
            k += 1  # Incrementar k solo después de procesar todos los lotes
        else:
            print(f"No se encontraron {k+1}-itemsets frecuentes. Terminando.")
            break
            
    return all_frequent_itemsets, support_dict

In [17]:
#Funcion para generar reglas
def generar_reglas(itemsets_frecuentes, support_dict, min_confidence=0.5):
    rules = []

    for k in range(2, len(itemsets_frecuentes) + 1):
        if k not in itemsets_frecuentes:
            continue
        for itemset in itemsets_frecuentes[k]:
            itemset_support = support_dict[tuple(itemset)]
            for i in range(len(itemset)):
                from itertools import combinations
                for antecedent_items in combinations(itemset, i):
                    antecedent = list(antecedent_items)
                    consequent = [item for item in itemset if item not in antecedent]
                    
                    # soporte antecedente
                    antecedent_support = support_dict[tuple(antecedent)]
                    
                    # Confianzita
                    confidence = itemset_support / antecedent_support
                    
                    # Si cumple con la confianza mínima, se agregar
                    if confidence >= min_confidence:
                        # lift
                        consequent_support = support_dict[tuple(consequent)]
                        lift = confidence / consequent_support
                        
                        rules.append({
                            'antecedent': antecedent,
                            'consequent': consequent,
                            'support': itemset_support,
                            'confidence': confidence,
                            'lift': lift
                        })
    
    
    if rules:
        rulesDf = pd.DataFrame(rules)

        #lift descendente
        rulesDf = rulesDf.sort_values('lift', ascending=False).reset_index(drop=True)
        return rulesDf
    else:
        return pd.DataFrame(columns=['antecedent', 'consequent', 'support', 'confidence', 'lift'])

In [18]:
#aplicacion en todo el algoritmo
transactions_list = sparse_transaction(order_matrix_sparse, item_mapping)

# Verificar el resultado
print(f"Número de transacciones en transactions_list: {len(transactions_list)}")
if len(transactions_list) > 0:
    print("Primeras 3 transacciones:")
    for i in range(min(3, len(transactions_list))):
        print(f"Transacción {i} (longitud {len(transactions_list[i])}): {transactions_list[i][:5]}")

#Verificacion vacias
empty_transactions = sum(1 for t in transactions_list if len(t) == 0)
print(f"Transacciones vacías: {empty_transactions} de {len(transactions_list)}")

#distribucion frecuencias items
item_freq = {}
for transaction in transactions_list:
    for item in transaction:
        item_freq[item] = item_freq.get(item, 0) + 1

#organizar por frecuencia
sorted_items = sorted(item_freq.items(), key=lambda x: x[1], reverse=True)

#y vreificamos cuantos superarin diferentes umbrales
print("Top 10 items más frecuentes:")
for item, count in sorted_items[:10]:
    support = count / len(transactions_list)
    print(f"Item: {item}, Frecuencia: {count}, Soporte: {support:.6f}")

Número de transacciones en transactions_list: 131209
Primeras 3 transacciones:
Transacción 0 (longitud 8): ['Bag of Organic Bananas', 'Bulgarian Yogurt', 'Cucumber Kirby', 'Lightly Smoked Sardines in Olive Oil', 'Organic 4% Milk Fat Whole Milk Cottage Cheese']
Transacción 1 (longitud 8): ['Asparagus', 'Cage Free Extra Large Grade AA Eggs', 'Grated Pecorino Romano Cheese', 'Organic Garnet Sweet Potato (Yam)', 'Organic Half & Half']
Transacción 2 (longitud 9): ['Bunched Cilantro', 'Flat Parsley, Bunch', 'Fresh Dill', 'Green Peas', 'Organic Baby Arugula']
Transacciones vacías: 0 de 131209
Top 10 items más frecuentes:
Item: Banana, Frecuencia: 18726, Soporte: 0.142719
Item: Bag of Organic Bananas, Frecuencia: 15480, Soporte: 0.117980
Item: Organic Strawberries, Frecuencia: 10894, Soporte: 0.083028
Item: Organic Baby Spinach, Frecuencia: 9784, Soporte: 0.074568
Item: Large Lemon, Frecuencia: 8135, Soporte: 0.062000
Item: Organic Avocado, Frecuencia: 7409, Soporte: 0.056467
Item: Organic Has

In [19]:
print(f"Número de transacciones en transactions_list: {len(transactions_list)}")
print("Muestra de transacciones:")
for i in range(min(5, len(transactions_list))):
    print(f"Transacción {i}: {transactions_list[i][:5]}...")

Número de transacciones en transactions_list: 131209
Muestra de transacciones:
Transacción 0: ['Bag of Organic Bananas', 'Bulgarian Yogurt', 'Cucumber Kirby', 'Lightly Smoked Sardines in Olive Oil', 'Organic 4% Milk Fat Whole Milk Cottage Cheese']...
Transacción 1: ['Asparagus', 'Cage Free Extra Large Grade AA Eggs', 'Grated Pecorino Romano Cheese', 'Organic Garnet Sweet Potato (Yam)', 'Organic Half & Half']...
Transacción 2: ['Bunched Cilantro', 'Flat Parsley, Bunch', 'Fresh Dill', 'Green Peas', 'Organic Baby Arugula']...
Transacción 3: ['Organic Blueberries', 'Organic Cucumber', 'Organic Grape Tomatoes', 'Organic Pomegranate Kernels', 'Organic Raspberries']...
Transacción 4: ['100% Organic Unbleached All-Purpose Flour', 'Aluminum Foil', 'Baby Swiss Slices Cheese', 'Bag of Organic Bananas', 'Black Beans']...


In [20]:
# verificacion para observar distintos umbrales
thresholds = [0.1, 0.05, 0.01, 0.005]
for threshold in thresholds:
    items_above = sum(1 for _, count in item_freq.items() if count/len(transactions_list) >= threshold)
    print(f"Items con soporte >= {threshold}: {items_above}")

min_support = 0.01  # Reducir el umbral de soporte en caso de(se redujo al minimo posible para ver si funciona)

start_time = time.time()
frequent_itemsets, support_dict = apriori_lotes(
    transactions_list, 
    min_support=min_support,
    batch_size=100000
)
apriori_time = time.time() - start_time

Items con soporte >= 0.1: 2
Items con soporte >= 0.05: 7


Items con soporte >= 0.01: 104
Items con soporte >= 0.005: 256
Procesando 131209 transacciones con soporte minimo 0.01
Fase 1: Generando 1-itemsets frecuentes
Procesando lote 1/ 2 (transacciones 0-100000)
Procesando lote 2/ 2 (transacciones 100000-131209)
  Se encontraron 104 1-itemsets frecuentes
Fase 2: Generando 2-itemsets frecuentes
Generados 5356 candidatos, 5356 despues de poda
Procesando lote 1/2 (transacciones 0-100000)
  Progreso: 10000/100000 transacciones procesadas
  Progreso: 20000/100000 transacciones procesadas
  Progreso: 30000/100000 transacciones procesadas
  Progreso: 40000/100000 transacciones procesadas
  Progreso: 50000/100000 transacciones procesadas
  Progreso: 60000/100000 transacciones procesadas
  Progreso: 70000/100000 transacciones procesadas
  Progreso: 80000/100000 transacciones procesadas
  Progreso: 90000/100000 transacciones procesadas
  Progreso: 100000/100000 transacciones procesadas
Procesando lote 2/2 (transacciones 100000-131209)
  Progreso: 10000

In [21]:
#itemsets frecuentes (resumen)
print("\nResumen de itemsets frecuentes:")
for k, itemsets in frequent_itemsets.items():
    print(f"  {k}-itemsets: {len(itemsets)}")


Resumen de itemsets frecuentes:
  1-itemsets: 104
  2-itemsets: 16
